# Awk - exercice 2 - éléments de correction

L'objectif de cet exercice est de vous faire analyser les données d'un fichier tabulé avec `awk`.

## Préparation des données

Supprimez tout d'abord les éventuels fichiers résiduels :

In [1]:
rm -f mng2015.csv mng2015_children_malaria_codebook.txt

Téléchargez le fichier `mng2015.csv` :

In [2]:
wget https://zenodo.org/record/154453/files/mng2015_children_malaria_data.csv -O mng2015.csv

--2020-09-17 22:15:35--  https://zenodo.org/record/154453/files/mng2015_children_malaria_data.csv
Résolution de zenodo.org (zenodo.org)… 188.184.117.155
Connexion à zenodo.org (zenodo.org)|188.184.117.155|:443… connecté.
requête HTTP transmise, en attente de la réponse… 200 OK
Taille : 19283 (19K) [text/plain]
Enregistre : «mng2015.csv»

mng2015.csv         100%[===================>]  18,83K  --.-KB/s    ds 0,03s   

2020-09-17 22:15:36 (634 KB/s) - «mng2015.csv» enregistré [19283/19283]



Téléchargez également le *codebook* associé :

In [3]:
wget https://zenodo.org/record/154453/files/mng2015_children_malaria_codebook.txt

--2020-09-17 22:15:36--  https://zenodo.org/record/154453/files/mng2015_children_malaria_codebook.txt
Résolution de zenodo.org (zenodo.org)… 188.184.117.155
Connexion à zenodo.org (zenodo.org)|188.184.117.155|:443… connecté.
requête HTTP transmise, en attente de la réponse… 200 OK
Taille : 1953 (1,9K) [text/plain]
Enregistre : «mng2015_children_malaria_codebook.txt»

mng2015_children_ma 100%[===================>]   1,91K  --.-KB/s    ds 0s      

2020-09-17 22:15:36 (810 MB/s) - «mng2015_children_malaria_codebook.txt» enregistré [1953/1953]




## Analyses

Ce [jeu de données](https://zenodo.org/record/154453/) a été généré dans le cadre de l'étude :

*Plasmodium falciparum infection in febrile Congolese children: prevalence of clinical malaria 10 years after introduction of artemisinin‐combination therapies*,  
publié dans *Tropical Medicine & International Health*, 2016, DOI [10.1111/tmi.1278](https://doi.org/10.1111/tmi.12786). Une version PDF de l'article est consultable [ici](http://cupnet.net/docs/Etoka-Beka_2016_TMIH.pdf).

Cette étude portaient sur la prévalence du paludisme chez des enfants de 1 à 10 ans, à l’hôpital de Brazzaville, en République du Congo.


### Format du jeu de données

Ce jeu  de données à deux particularités :

1. La première ligne contient l'entête, c'est-à-dire le nom des colonnes. Les données se trouvent donc au delà de la première ligne. Il faut dire à `awk` d'ignorer cette première ligne (avec la variable `NR`).
2. Le jeu de données est au format *comma-separated values* (`.csv`), ce qui signifie que les différentes colonnes sont séparées par des virgules. Il faut expressément dire à `awk` que le séparateur de colonne est une virgule (avec l'option `-F`).

### Codebook

Pour comprendre ce jeu de données, prenez connaissance du *codebook*. Le *codebook* explique la signification de chaque colonne du jeu de données.

Affichez les 15 premières lignes du *codebook* avec la commande `head` :

In [4]:
head -n 15 mng2015_children_malaria_codebook.txt

patient_id
    - patient id
    - format: MNGXXX with XXX a sequential number
    - values: MNG001, MNG270 ...

sampling_date
    - date of blood sampling
    - format: YYYY/MM/DD
    - values: 2015/02/04, 2014/12/10 ...

age
    - age of patient (in years)
    - format: integer
    - values: 4, 10 ...



### Méthode de lecture avec `awk`

Pour que le jeu de données soit lu correctement, les commandes avec `awk` débuteront de la sorte :
```
$ awk -F "," 'NR>1 ...
```
Pour rappel, `-F ","` indique à `awk` que les colonnes sont séparées par une virgule et `NR>1` saute la première ligne (qui contient les noms des colonnes).

### Sexe des patients

En utilisant les outils `awk` et `wc`, déterminez :

1. Le nombre de patients de sexe masculin.

In [5]:
awk -F "," 'NR>1 && $4 ~ /^male/' mng2015.csv | wc -l

124


2. Le nombre de patients de sexe féminin.

In [6]:
awk -F "," 'NR>1 && $4 ~ /^female/' mng2015.csv | wc -l

105


3. Vérifiez que ces valeurs concordent avec le nombre total de patients.

In [7]:
awk -F "," 'NR>1' mng2015.csv | wc -l

229


4. Pourquoi la commande `wc -l mng2015.csv` ne donne pas le nombre total de patients ?

`wc` seul ne donne pas le bon résultat car la première ligne (la ligne avec l'entête des colonnes) est aussi prise en compte.

### Âge moyen

En utilisant `awk` uniquement, déterminez :

1. L'âge moyen de tous les patients.

In [8]:
awk -F "," 'NR>1  {age+=$3; count++} END {print "mean age:", age/count}' mng2015.csv

mean age: 3.0917


Suivant votre système d'exploitation, vous pouvez obtenir `3,0917`. Aucune importance pour le moment mais gardez cela en tête pour la suite.

2. L'âge moyen des patients ayant un profil hémoglobinique normal (colonne *hb_profile* avec le profile *normal hemoglobin*). Attention, utilisez le *codebook* pour savoir comment est représentée l'information *hb_profile*.

Il faut aller chercher dans le champ *hb_profile* (7e champ) qui contient le profil hémoglobinique des patients. Les patients avec profil hémoglobinique normal sont notés *AA*. Les patients avec un profil hémoglobinique drépanocytaire sont notés *AS*.

In [9]:
awk -F "," 'NR>1 && $7~/AA/ {age+=$3; count++} END {print "mean age:", age/count}' mng2015.csv

mean age: 3.12563


Suivant votre système d'exploitation, vous pouvez obtenir `3,12563`.

3. L'âge moyen des patients ayant un profil hémoglobinique drépanocytaire (colonne *hb_profile* avec le profile *sickle cell trait*).

In [10]:
awk -F "," 'NR>1 && $7~/AS/ {age+=$3; count++} END {print "mean age:", age/count}' mng2015.csv

mean age: 2.86667


Suivant votre système d'exploitation, vous pouvez obtenir `2,86667`.

4. Comparez vos résultats avec ceux de l'article scientifique dont sont tirées ces données.

L'article est disponible [ici](http://cupnet.net/docs/Etoka-Beka_2016_TMIH.pdf). On obtient des valeurs en accord avec celles du tableau 1 de l'article.

### Taux d'hémoglobine moyen

Le taux d'hémoglobine se trouve dans la colonne *hb_conc* (8e colonne).

En utilisant `awk` uniquement, déterminez :

1. Le taux d'hémoglobine (colonne *hb_conc*) moyen des patients ayant un profil hémoglobinique normal (codé par `AA`).

In [11]:
awk -F "," 'NR>1 && $7~/AA/ {hb+=$8; count++} END {print "mean hb:", hb/count}' mng2015.csv

mean hb: 11.3447


Suivant votre système d'exploitation, vous pouvez obtenir :
```
mean hb: 10,8643
```
ce qui n'est pas du tout la valeur attendue.

On remarque par contre que la valeur décimale `10,8643` est exprimée avec le symbole `,` comme séparateur décimale alors que c'est le symbole `.` qui est utilisé dans le jeu de données.

Ce problème est récurrent sur les systèmes informatiques configurés en français. Pour corriger cela, il suffit de déclarer au préalable la variable d'environnement `LC_NUMERIC=C` avec la commande :
```
export LC_NUMERIC=C
```
Cette commande n'est à lancer qu'une seule fois, avant les analyses, ou à mettre dans votre fichier de configuration du shell (`.bashrc` ou `.profile`).

Restez toujours vigilants sur ce problème de séparateur décimal (`.` ou `,`).

2. Le taux d'hémoglobine (colonne *hb_conc*) moyen des patients drépanocytaires (codé par `AS`).

In [12]:
awk -F "," 'NR>1 && $7~/AS/ {hb+=$8; count++} END {print "mean hb:", hb/count}' mng2015.csv

mean hb: 11.18


3. Comparez vos résultats avec ceux de l'article scientifique dont sont tirées ces données.

Les résultats sont bien en accords avec le tableau 1 de l'article.

### Température moyenne

La température se trouve dans le champ *temp* (6e colonne).

La méthode de diagnostic du paludisme se trouve dans le champ *malaria* (12e colonne). Les valeurs possibles pour ce champ sont : 
- *negative* (pas de paludisme), 
- *uncomplicated* (paludisme diagnostiqué par microcopie)
- et *submicroscopic* (paludisme diagnostiqué par *Polymerase Chain Reaction* (PCR)).

En utilisant `awk` uniquement, déterminez :

1. La température (colonne *temp*) moyenne des patients ayant un paludisme diagnostiqué par microscopie (colonne *malaria* correspondante à *malaria diagnosed by microscopy*). Attention, utilisez le *codebook* pour savoir comment est représentée l'information *malaria*.

On sélectionne les patients dont le champ *malaria* vaut *uncomplicated*.

In [13]:
awk -F "," 'NR>1 && $12~/uncomplicated/ {temp+=$6; count++} END {print "mean temp:", temp/count}' mng2015.csv

mean temp: 38.7727


2. La température (colonne *temp*) moyenne des patients ayant un paludisme diagnostiqué par *polymerase chain reaction* (colonne *malaria* correspondante à *malaria diagnosed by PCR*).

On sélectionne les patients dont le champ *malaria* vaut *submicroscopic*.

In [14]:
awk -F "," 'NR>1 && $12~/submicroscopic/ {temp+=$6; count++} END {print "mean temp:", temp/count}' mng2015.csv

mean temp: 37.7388


3. Comparez vos résultats avec ceux de l'article scientifique dont sont tirées ces données.

Les résultats sont bien en accords avec le tableau 2 de l'article.